In [11]:
import datetime as dt
import pandas as pd
import requests
from prefect.blocks.system import Secret
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session
from mcpdb.tables import Asset, Provider, ProviderAsset, ProviderAssetMarket

In [9]:
postgresql_password: str = (await Secret.load("postgresql-password")).get()
host = "db-postgresql-lon1-65351-do-user-18535103-0.m.db.ondigitalocean.com"
port = 25060
database = "defaultdb"
user = "doadmin"
url = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user=user,
    password=postgresql_password,
    host=host,
    port=port,
    database=database,
)
engine = create_engine(url)

In [3]:
categories: list[str] = []
with Session(engine) as session:
    # Get the coin desk provider.
    stmt = select(Provider).where(Provider.name == "CoinDesk")
    provider = session.execute(stmt).scalar_one_or_none()

    # Get all categories from provider assets.
    if provider:
        stmt = select(ProviderAsset).where(ProviderAsset.provider_id == provider.id)
        provider_assets = session.execute(stmt).scalars().all()
        for provider_asset in provider_assets:
            if provider_asset.asset_code:
                categories.append(provider_asset.asset_code)

categories

['BTC', 'ETH', 'XRP', 'BNB']

In [4]:
host = "https://data-api.coindesk.com"
uri = "/news/v1/article/list"
params = {
    "lang": "EN",
    "limit": 100,
    "to_ts": (dt.datetime.now() - dt.timedelta(hours=2)).timestamp(),
    "categories": ",".join(categories),
    "source_ids": ",".join([provider.name.lower()]),
}
response = requests.get(f"{host}{uri}", params=params)
if response.status_code != 200:
    raise Exception(f"Failed to fetch data from Coindesk API: {response.status_code}")
data = response.json()["Data"]
data

[{'TYPE': '121',
  'ID': 46723393,
  'GUID': '1e58e1c1-c23f-434b-be83-fe82eb2ca204',
  'PUBLISHED_ON': 1750813912,
  'PUBLISHED_ON_NS': None,
  'IMAGE_URL': 'https://resources.cryptocompare.com/news/5/46723393.jpeg',
  'TITLE': 'Asia Morning Briefing: Analysts Say BTC’s Long-Term Focus Is Easing War Jitters',
  'SUBTITLE': "PLUS: Tim Draper sees parallels between the flight to BTC and Microsoft's early days.",
  'AUTHORS': 'Sam Reynolds,AI Boost',
  'URL': 'https://www.coindesk.com/markets/2025/06/24/asia-morning-briefing-analysts-say-btcs-long-term-focus-is-easing-war-jitters',
  'SOURCE_ID': 5,
  'BODY': 'Good Morning, Asia. Here\'s what\'s making news in the markets: Welcome to Asia Morning Briefing, a daily summary of top stories during U.S. hours and an overview of market moves and analysis. For a detailed overview of U.S. markets, see CoinDesk\'s Crypto Daybook Americas. After a tense weekend that saw the U.S. bomb an Iranian nuclear site , bitcoin btc has regained its footing, h

In [6]:
df = pd.DataFrame(data)
sample = data[0]
for col in df.columns:
    print(f"Column: {col}, Type: {df[col].dtype}, Sample: {sample[col]}")

Column: TYPE, Type: object, Sample: 121
Column: ID, Type: int64, Sample: 46723393
Column: GUID, Type: object, Sample: 1e58e1c1-c23f-434b-be83-fe82eb2ca204
Column: PUBLISHED_ON, Type: int64, Sample: 1750813912
Column: PUBLISHED_ON_NS, Type: object, Sample: None
Column: IMAGE_URL, Type: object, Sample: https://resources.cryptocompare.com/news/5/46723393.jpeg
Column: TITLE, Type: object, Sample: Asia Morning Briefing: Analysts Say BTC’s Long-Term Focus Is Easing War Jitters
Column: SUBTITLE, Type: object, Sample: PLUS: Tim Draper sees parallels between the flight to BTC and Microsoft's early days.
Column: AUTHORS, Type: object, Sample: Sam Reynolds,AI Boost
Column: URL, Type: object, Sample: https://www.coindesk.com/markets/2025/06/24/asia-morning-briefing-analysts-say-btcs-long-term-focus-is-easing-war-jitters
Column: SOURCE_ID, Type: int64, Sample: 5
Column: BODY, Type: object, Sample: Good Morning, Asia. Here's what's making news in the markets: Welcome to Asia Morning Briefing, a dail